# Skeletonization of volumes

## Pipeline
- Open `NRRD` volume files with Fiji.
- Visualize volume using Fiji 3D viewer.
- Run `Skeletonize` plugin.
- Run `Analyze Skeleton` plugin (tick boxes for full output).
- Export skeleton info as `CSV` files.
- Run code in this notebook.

## Some additional doc

### Convert `.mha` to `.nrrd`
- Open `.mha` file with Fiji.
- Convert to 8-bit format.
    - Image -> Type -> 8-bit
- Normalize histogram (_ie_ mesh values set to 255).
    - Process -> Enhance contrast -> tick normalize, process all slices, use stack histogram
- Export as `.nrrd`.
- Exported `NRRD` files available in Box folder `tommaso_data`.

### About skeletonization algorithm
- from Lee et al. 1994 900+ citations [paper](http://155.98.16.52/devbuilds/biomesh3d/FEMesher/references/lee94-3dskeleton.pdf).
- Implemented in ITK (Homann 2007) [link](http://www.sci.utah.edu/~dmw/FEMesher/references/ITKbinaryThinningImageFilter3D.pdf)
- Implemented in Fiji. Plugin `Skeletonize3D` (last update 2017) [link](https://imagej.net/Skeletonize3D).
- Implemented in (Python module) `skelet3d` [link](https://pypi.org/project/skelet3d/1.5.13/).
- Implemented in MATLAB implementation [link](https://github.com/phi-max/skeleton3d-matlab).
- Implemented in `scikit-image` as `skeletonize_3d` from scikit-image [doc](https://scikit-image.org/docs/dev/auto_examples/edges/plot_skeleton.html).

In [1]:
import ipyvolume as ipv
import ipyvolume.pylab as p3
import nrrd
import re
import tlib.lung as tlu  # tommaso library - lung project
import warnings
warnings.filterwarnings('ignore')

***
## Load data and visualize segmentation

Load `slicer_test` segmentation and analysis. Replace paths w your own.

In [2]:
# Load segmentation
slicer_data_path = os.path.join(home, 'datasets/lung/slicer/on_sample_ct_data/airways_seg_int8.nrrd')
assert os.path.isfile(slicer_data_path)
slicer_data, slicer_data_info = nrrd.read(slicer_data_path)

# Load skeleton analysis
slicer_skeleton_path = os.path.join(home, 'datasets/lung/slicer/on_sample_ct_data/skeleton_analysis.csv')
assert os.path.isfile(slicer_skeleton_path)
slicer_skeleton = tlu.Skeleton(slicer_skeleton_path)
# slicer_df = pd.read_csv(skeleton_path)

# Visualize segmentation
# ipv.quickvolshow(slicer_data, opacity=[.2, 0, 0], level=[.21, 1, .9])

Same for `D175` dataset.

In [4]:
# Load segmentation
d175_data_path = os.path.join(home, 'datasets/lung/segmentation/D175_segmented.nrrd')
assert os.path.isfile(d175_data_path)
d175_data, d175_data_info = nrrd.read(d175_data_path)

# Load skeleton analysis
d175_skeleton_path = os.path.join(home, 'datasets/lung/segmentation/D175_skeleton.csv')
assert os.path.isfile(d175_skeleton_path)
d175_skeleton = tlu.Skeleton(d175_skeleton_path)
# d175_df = pd.read_csv(d175_skeleton_path)

# Visualize segmentation
# ipv.quickvolshow(slicer_data, opacity=[.2, 0, 0], level=[.21, 1, .9])

Visualize data to fine tune `opacity`, `level` and `xyzlim` for each dataset.

In [5]:
slicer_info = {
    'opacity': [.7, 0, 0],
    'level': [.2, 1, .9],
    'level_width': .08,
    'max_opacity': .7,
    'xyz_lim': 500,
}
# ipv.figure()
# ipv.volshow(slicer_data, opacity=slicer_info['opacity'],  level=slicer_info['level'],
#             level_width=slicer_info['level_width'],  
#             max_opacity=slicer_info['max_opacity'])
# ipv.xyzlim(0, slicer_info['xyz_lim'])
# ipv.show()

In [6]:
d175_info = {
    'opacity': [.7, 0, 0],
    'level': [.2, 1, .9],
    'level_width': .08,
    'max_opacity': .7,
    'xyz_lim': 1000,
}
# ipv.figure()
# ipv.volshow(d175_data, opacity=d175_info['opacity'],  level=d175_info['level'],
#             level_width=d175_info['level_width'],  
#             max_opacity=d175_info['max_opacity'])
# ipv.xyzlim(0, d175_info['xyz_lim'])
# ipv.show()

***
## Show results

In [2]:
def show_figure(data, info, skeleton):
    """Helper that overlays ending/branching points to segmentation"""
    sphere_size = .8
    ipv.figure()
    ipv.volshow(data, opacity=info['opacity'], 
                level=info['level'],
                level_width=info['level_width'],  
                max_opacity=info['max_opacity'])

    xs_1, ys_1, zs_1 = skeleton.get_points_w_connectivity(1)
    ipv.scatter(zs_1, ys_1, xs_1, marker='sphere', color='blue', size=sphere_size)

    xs_3, ys_3, zs_3 = skeleton.get_points_w_connectivity(3)
    ipv.scatter(zs_3, ys_3, xs_3, marker='sphere', color='yellow', size=sphere_size)

    ipv.xyzlim(0, info['xyz_lim'])
    ipv.show()

In [4]:
show_figure(data=slicer_data, info = slicer_info, skeleton = slicer_skeleton)

show_figure(data=d175_data, info = d175_info, skeleton = d175_skeleton)

***
## Where do we go from here

- Andrea Tagliasacchi: expert in skeletonization. Method [paper](https://ialhashim.github.io/files/documents/tag_sgp12.pdf), perhaps works better than ours? Implemented in [obscure framework](https://github.com/ataiya/starlab-mcfskel) and [CGAL](https://doc.cgal.org/latest/Surface_mesh_skeletonization/index.html) by student.
- [SkeletonLab](http://francescousai.info/skel_lab.html), software for easy editing of skeletons. Learn how to export our skeleton in right format then edit it using their framework?